In [5]:
!pip install imutils
!pip install winsound

  Using cached imutils-0.5.4-py3-none-any.whl


ERROR: Could not find a version that satisfies the requirement winsound (from versions: none)
ERROR: No matching distribution found for winsound


In [7]:
import cv2
import imutils
import winsound
import threading

In [7]:
import cv2
import imutils
import threading
import winsound  
import time

cv2.namedWindow("window", cv2.WINDOW_NORMAL)
cv2.resizeWindow("window", 640, 480)
cap = cv2.VideoCapture(0)
res, start_frame = cap.read()
start_frame = imutils.resize(start_frame, width=500)
start_frame = cv2.cvtColor(start_frame, cv2.COLOR_BGR2GRAY)
start_frame = cv2.GaussianBlur(start_frame, (21, 21), 0)

alarm = False
alarm_mode = False
alarm_counter = 0
frame_count = 0  

def beep_alarm():
    global alarm
    for i in range(5):  
        if not alarm_mode:
            break
        print("ALARM!")
        winsound.Beep(2500, 1000)
    alarm = False

while True:
    res, frame = cap.read()
    frame = imutils.resize(frame, width=500)
    if alarm_mode:
        frame_bw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_bw = cv2.GaussianBlur(frame_bw, (21, 21), 0)
        difference = cv2.absdiff(frame_bw, start_frame)
        threshold = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)[1]
        frame_count += 1
        if frame_count % 100 == 0:  
            start_frame = frame_bw
        if threshold.sum() > 300:  
            alarm_counter += 1
        else:
            alarm_counter = max(0, alarm_counter - 1)  

        cv2.imshow("window", threshold)
    else:
        cv2.imshow("window", frame)

    if alarm_counter > 20 and not alarm:
        alarm = True
        threading.Thread(target=beep_alarm).start()

    key = cv2.waitKey(1)
    if key == ord("t"):
        alarm_mode = not alarm_mode
        alarm_counter = 0
    if key == ord("q"):
        alarm_mode = False
        break
    time.sleep(1/30)  

cap.release()
cv2.destroyAllWindows()
